In [24]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, dash_table
import webbrowser
from threading import Timer
from datetime import timedelta

# Charger les données
data = pd.read_csv(r'C:\Users\liamh\OneDrive\Documents\0_3A\Rapport essai\10122024\J2KDL-97.CSV', encoding='latin1')

# Nettoyer les données
data_cleaned = data.iloc[1:].copy()
requested_columns = ['T.Gas', 'CO2', 'Eff.', 'COr']
for col in requested_columns:
    data_cleaned[col] = pd.to_numeric(data_cleaned[col], errors='coerce')
data_cleaned['Datetime'] = pd.to_datetime(data_cleaned['Date'] + ' ' + data_cleaned['Time'], errors='coerce')

# Application Dash
app = Dash(__name__)

# Layout de l'application
app.layout = html.Div([
    html.H1("Dynamic Multi-Axis Annotations with Zoom", style={'color': 'white', 'textAlign': 'center'}),
     # Section pour afficher la durée sélectionnée
    html.Div(id='selected-duration', style={
        'color': 'white',
        'fontSize': '18px',
        'margin': '10px',
        'textAlign': 'center'
    }),
    dcc.Graph(id='dynamic-graph', config={'scrollZoom': True}, style={'backgroundColor': 'black'}),
    html.Div([
        dash_table.DataTable(
            id='stats-table',
            style_table={'marginTop': 20, 'overflowX': 'auto'},
            style_header={
                'backgroundColor': 'black',
                'color': 'white',
                'fontWeight': 'bold',
                'fontSize': '18px',  # Plus gros pour les titres
                'textAlign': 'center'
            },
            style_cell={
                'backgroundColor': 'black',
                'color': 'white',
                'textAlign': 'center',
                'fontSize': '16px'
            },
            style_data_conditional=[  # Styles conditionnels
                {
                    'if': {
                        'filter_query': '{Parameter} = "Eff." && {Mean} < 75',
                        'column_id': 'Mean'
                    },
                    'color': 'red',
                    'fontWeight': 'bold'
                },
                {
                    'if': {
                        'filter_query': '{Parameter} = "Eff." && {Mean} >= 75',
                        'column_id': 'Mean'
                    },
                    'color': 'green',
                    'fontWeight': 'bold'
                },
                {
                    'if': {
                        'filter_query': '{Parameter} = "COr" && {Mean} < 1500',
                        'column_id': 'Mean'
                    },
                    'color': 'green',
                    'fontWeight': 'bold'
                },
                {
                    'if': {
                        'filter_query': '{Parameter} = "COr" && {Mean} >= 1500',
                        'column_id': 'Mean'
                    },
                    'color': 'red',
                    'fontWeight': 'bold'
                }
            ]
        )
    ])
], style={'backgroundColor': 'black', 'padding': '20px'})


# Callback pour mettre à jour le graphique et les annotations dynamiquement
@app.callback(
    [Output('dynamic-graph', 'figure'),
     Output('stats-table', 'data'),
     Output('stats-table', 'columns'),
     Output('selected-duration', 'children')],
    [Input('dynamic-graph', 'relayoutData')]
)
def update_graph(relayoutData):
    # Définir la plage par défaut
    start_time = data_cleaned['Datetime'].min()
    end_time = data_cleaned['Datetime'].max()

    # Ajuster en fonction du zoom
    if relayoutData and 'xaxis.range[0]' in relayoutData:
        start_time = pd.to_datetime(relayoutData['xaxis.range[0]'])
        end_time = pd.to_datetime(relayoutData['xaxis.range[1]'])

    # Calculer la durée sélectionnée au format hh:mm:ss
    duration = end_time - start_time
    duration_formatted = str(timedelta(seconds=int(duration.total_seconds())))
    duration_text = f"Selected Duration: {duration_formatted}" if duration.total_seconds() > 0 else "Full Duration Selected"

    # Filtrer les données
    filtered_data = data_cleaned[(data_cleaned['Datetime'] >= start_time) & 
                                 (data_cleaned['Datetime'] <= end_time)]

   # Calcul des statistiques
    stats = [{'Parameter': col, 
              'Min': f"{filtered_data[col].min():.2f}", 
              'Max': f"{filtered_data[col].max():.2f}", 
              'Mean': f"{filtered_data[col].mean():.2f}"} 
             for col in requested_columns]

    # Configuration des axes et tracés
    fig = go.Figure()
    colors = {'T.Gas': 'blue', 'CO2': 'red', 'Eff.': 'green', 'COr': 'purple'}

    # Ajouter chaque série sur un axe différent
    fig.add_trace(go.Scatter(
        x=filtered_data['Datetime'], y=filtered_data['T.Gas'],
        mode='lines', name='T.Gas', line=dict(color='blue'),
        yaxis='y1'
    ))
    fig.add_trace(go.Scatter(
        x=filtered_data['Datetime'], y=filtered_data['CO2'],
        mode='lines', name='CO2', line=dict(color='red'),
        yaxis='y2'
    ))
    fig.add_trace(go.Scatter(
        x=filtered_data['Datetime'], y=filtered_data['Eff.'],
        mode='lines', name='Eff.', line=dict(color='green'),
        yaxis='y3'
    ))
    fig.add_trace(go.Scatter(
        x=filtered_data['Datetime'], y=filtered_data['COr'],
        mode='lines', name='CO 13%O2', line=dict(color='purple'),
        yaxis='y4'
    ))

    # Configuration des axes indépendants
    fig.update_layout(
    title='T.Gas, CO2, Efficiency, and CO 13%02',
    xaxis=dict(title='Time'),

    yaxis=dict(
        title='T.Gas [°C]',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
    ),
    yaxis2=dict(
        title='CO2 [%]',
        titlefont=dict(color='red'),
        tickfont=dict(color='red'),
        side='right',
        overlaying='y'
    ),
    yaxis3=dict(
        title='Efficiency [%]',
        titlefont=dict(color='green'),
        tickfont=dict(color='green'),
        overlaying='y',
        anchor='free',
        autoshift=True
    ),
    yaxis4=dict(
        title='CO 13%02 [ppm]',
        titlefont=dict(color='purple'),
        tickfont=dict(color='purple'),
         overlaying='y',
        anchor='free',
        autoshift=True
    ),
    legend=dict(title='Parameters',orientation="v"),
    template='plotly_white'
    
    )
    # Ajouter la ligne fixe à 4% de CO2 (référentiel yaxis2)
    fig.add_trace(go.Scatter(
            x=[filtered_data['Datetime'].min(), filtered_data['Datetime'].max()],
            y=[4, 4],  # Ligne fixe à 4%
            mode='lines',
            name='CO2 (4%)',
            line=dict(color='red', dash='dash'),
            yaxis='y2',  # Associer la ligne à yaxis2
            hoverinfo='none'  # Désactiver le survol
        ))
    
    # Configuration du tableau
    columns = [{'name': col, 'id': col} for col in ['Parameter', 'Min', 'Max', 'Mean']]
    return fig, stats, columns, duration_text

# Fonction pour ouvrir automatiquement le navigateur
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

# Lancer l'application
if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True)


C:\Users\liamh\AppData\Local\Temp\ipykernel_23132\695614410.py:9: DtypeWarning:

Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.

